# ResNet18 Model for Image Classification

In [ ]:
# Importing the libraries
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from PIL import Image
import cv2, glob
import random
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
class Weather(Dataset):
    def __init__(self, folder):
        dew = glob(folder+'/dew/*.jpg')
        fogsmog = glob(folder+'/fogsmog/*.jpg')
        frost = glob(folder+'/frost/*.jpg')
        glaze = glob(folder+'/glaze/*.jpg')
        hail = glob(folder+'/hail/*.jpg')
        lightning = glob(folder+'/lightning/*.jpg')
        rain = glob(folder+'/rain/*.jpg')
        rainbow = glob(folder+'/rainbow/*.jpg')
        rime = glob(folder+'/rime/*.jpg')
        sandstorm = glob(folder+'/sandstorm/*.jpg')
        snow = glob(folder+'/snow/*.jpg')

        # Accounting for class imbalance
        self.fpaths = dew + fogsmog + frost + glaze + hail + lightning + rain + rainbow + rime + sandstorm + snow

        # Normalizing the images
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        
        # Shuffling the images
        from random import shuffle, seed; seed(10); shuffle(self.fpaths)

        # Set targets
        self.targets = self.targets = [fpath.split('/')[-2] for fpath in self.fpaths]

    def __len__(self): return len(self.fpaths)

    def __getitem__(self, ix):
        f = self.fpaths[ix]
        target = self.targets[ix]
        im = (cv2.imread(f)[:,:,::-1])
        im = cv2.resize(im, (224,224))
        im = torch.tensor(im/255)
        im = im.permute(2,0,1)
        im = self.normalize(im) 
        return im.float().to(device), torch.tensor([target]).float().to(device)

In [ ]:
train = []
for line in os.open('train_images_labeled.csv'):
    train.append(line.split(',')[0])

test = []
for line in os.open('test_images_labeled.csv'):
    test.append(line.split(',')[0])

val = []
for line in os.open('val_images_labeled.csv'):
    val.append(line.split(',')[0])

train_data = Weather(train)

In [ ]:
im, label = train_data[200]
plt.imshow(im.permute(1,2,0).cpu())
print(label)

In [ ]:
def get_model():
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.fc = nn.Sequential(nn.Flatten(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid())
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)
    return model.to(device), loss_fn, optimizer

In [ ]:
!pip install torch_summary
from torchsummary import summary
model, criterion, optimizer = get_model()
summary(model, torch.zeros(1,3,224,224))